In [68]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import words
import csv,random

In [75]:
#parsing data

english_word_list = words.words()
random.shuffle(english_word_list)
english_word_list = list(map(lambda x:[x,0],english_word_list))

company_names = pd.read_csv("companies_sorted.csv",usecols = ["name"]).values.tolist()
company_names = list(map(lambda x:[x[0],1],company_names))[:250000]

company_names.extend(english_word_list)

inputs_labels = company_names
random.shuffle(inputs_labels)

for line in inputs_labels:
    try:
        line[0].lower()
    except:
        inputs_labels.remove(line)

    if len(line[0]) == 0:
        inputs_labels.remove(line)

TypeError: object of type 'float' has no len()

In [ ]:
X = list(map(lambda x:x[0].replace(""," "),inputs_labels))
Y = list(map(lambda x:x[1],inputs_labels))

In [ ]:
#*********************TOKENISER*********************

tkn = Tokenizer()
tkn.fit_on_texts(X)
total_chars = len(tkn.word_index) + 1

tokenised_list = tkn.texts_to_sequences(X)

maxSequenceLen = max([len(x) for x in tokenised_list])
inputSequences = np.array(pad_sequences(tokenised_list,maxlen = maxSequenceLen,padding = "pre"))

In [85]:
#*********************MODEL*********************

model = tf.keras.Sequential()

model.add(Embedding(total_chars,50,input_length = maxSequenceLen))
model.add(LSTM(64))
model.add(Dense(256,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))

model.compile(loss = "mse",optimizer = "adam")

In [86]:
tf.config.run_functions_eagerly(True)

model.fit(inputSequences,np.array(Y)epochs = 2,verbose = 1)

c:\Users\jayde\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


11513/13648 [========================>.....] - ETA: 14:25 - loss: 0.0897ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\Users\jayde\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\backend.py", line 5133, in <genexpr>
    ta.write(ta_index_to_write, out)  File "c:\Users\jayde\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),
13648/13648 [==============================] - 5490s 402ms/step - loss: 0.0861


In [94]:
tokenised_sequence = tkn.texts_to_sequences(["Ebay".replace(""," ")])

test_sequence = np.array(pad_sequences(tokenised_sequence,maxlen = maxSequenceLen,padding = "pre"))

model.predict(test_sequence)

1/1 [==============================] - 0s 243ms/step


array([[0.2661753]], dtype=float32)